In [1]:
import pickle
import numpy as np
import torch
from LogLikelihoodTorch import log_likelihood,Optimizationloop, Initialize
with open('.DataPhase.pickle','rb') as f:
    X = pickle.load(f) 
X_tensor = torch.from_numpy(X)
torch.set_default_dtype(torch.float64)


In [3]:
# Setting Up Parameters For Fitting of K Multivariate Watson Distributions

K = 7
p = 90

pi,kappa,mu = Initialize(p=90,K=7)
#%%
learning_rate = 0.2
n_iters = 20
epochs = 10

#torch.autograd.set_detect_anomaly(False)
Parameters = [
    {'params':pi},
    {'params':kappa},
    {'params':mu}
]

Adam = torch.optim.Adam(Parameters,lr=learning_rate)

In [4]:
Pi_fit,Kappa_fit,mu_fit = Optimizationloop(X_tensor[:,0:(330*10+1)],[pi,kappa,mu],lose=log_likelihood,Optimizer=Adam,n_iters=1)

epoch 1; Log-Likelihood = -241598.32495203137
